In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp 
import ast
import numpy as np

In [2]:
#CARGA DE ARCHIVOS

steam_games= pd.read_parquet("Data/steam_games.parquet")

user_items= pd.read_parquet("Data/items.parquet")

user_reviews= pd.read_parquet("Data/reviews.parquet")

In [7]:
#steam_games.head()
#user_items.head()
user_reviews.head()

,user_id,item_id,recommend,sentiment_analisis,posted_year
0,76561197970982479,1250,1,2,2011
1,76561197970982479,22200,1,2,2011
2,76561197970982479,43110,1,1,2011
3,js41637,251610,1,2,2014
4,js41637,227300,1,1,2013


In [8]:
steam_games.dtypes

genres          object
app_name        object
price          float64
item_id          Int64
developer       object
Año_estreno     object
dtype: object

In [9]:
user_items.dtypes

user_id             object
items_count          int64
item_id             object
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

In [10]:
#Transformo el tipo de datos de item_id a integer
user_items["item_id"] = user_items["item_id"].astype("Int64")

FUNCION #1: UserData

In [13]:
#Tomo las columnas relevantes de cada DF
cant_items= user_items[["user_id","item_id","items_count"]]

#Utilizo la columna "recommend" y su "user_id" del dataframe user_reviews
recommend= user_reviews[["user_id", "recommend"]]
# Y utilizo la columna "price" del dataframe steam_games , luego cambio el nombre de id por "item_id" a fin de poder realizar
# el merge
cant_dinero = steam_games[["price", "item_id"]] 


In [14]:
#Armo el DF final
df_funcion1= cant_items.merge(cant_dinero, on="item_id", how="left")


In [15]:
df_funcion1.head()

,user_id,item_id,items_count,price
0,76561197970982479,10,277,9.99
1,76561197970982479,20,277,4.99
2,76561197970982479,30,277,4.99
3,76561197970982479,40,277,4.99
4,76561197970982479,50,277,4.99


In [16]:
#Reviso  nulos
df_funcion1["price"].isnull().sum()

849274

In [17]:
# Defino una función para poder reemplazar los datos string de la columna "price" por flotantes 0.0

def cambio_a_float(valor):

    if pd.isna(valor): #esta parte de la función es para el caso de que no exista ningun string o float en el campo
        return 0.0
    try:
        flotante = float(valor) #si el valor es float lo conservo
        return flotante
    except (ValueError, TypeError): #si el valor es diferente retorno 0.0
        return 0.0
    
#Aplico esta función a la columna precio
df_funcion1["price"] = df_funcion1["price"].apply(cambio_a_float)

In [18]:
#Vuelvo a revisar para confimar que se hayan eliminado
df_funcion1["price"].isnull().sum()

0

In [19]:
#Defino la funcion #1 
"""
Parametro: 
        user_id(str) : ID del Usuario a consultar.
Retorna:
        user (dict): Información de un usuario ,
        -cantidad de dinero gastado (int): Dinero gastado por usuario
        -Porcentaje de recomendación usuario (float): Reviews realizadas por el usuario con respecto a la cantidad de 
        reviews poe usuario
        -cantidad de items (int):cantidad de juegos consumidos por usuario 
"""
def userdata(user_id):
    #Gasto por Usuario
    gasto = df_funcion1[df_funcion1["user_id"]== user_id]["price"].sum()
    
    #Cantidad de recomendaciones del usuario ingresado
    rec_user= recommend[recommend["user_id"]== user_id]["recommend"].sum()
    #Cantidad de recomendaciones totales por usuario
    total_recomendaciones= len(user_reviews["user_id"].unique())
    porcentaje=(rec_user/total_recomendaciones)*100
    
    #Cantidad de juegos que utilizo el usuario 
    count= df_funcion1[df_funcion1["user_id"]== user_id]["items_count"].iloc[0]
    return{
        "Cantidad de dinero gastado": float (gasto),
        "Porcentaje de recomendación usuario": round(float(porcentaje), 3),
        "Cantidad de items": int(count)
    }

In [20]:
#Pruebo la función:
user_id= "76561198075035785"
userdata(user_id)

{'Cantidad de dinero gastado': 39.98,
 'Porcentaje de recomendación usuario': 0.0,
 'Cantidad de items': 3}

In [21]:
#CHECKEO con usuario de poco items
check = df_funcion1['user_id'].value_counts()
print (check)
print (check.index[65000])

user_id
phrostb              17007
thugnificent         14765
chidvd               13799
piepai               13184
mayshowganmore       11712
                     ...  
funnychat1               1
autogear                 1
76561198085831299        1
76561198067630449        1
bills4ded                1
Name: count, Length: 70912, dtype: int64
76561198045882569


In [22]:
df_funcion1[df_funcion1["user_id"] == "76561198075035785"]

,user_id,item_id,items_count,price
10070065,76561198075035785,240,3,19.99
10070066,76561198075035785,550,3,19.99
10070067,76561198075035785,223530,3,0.00


In [23]:
#Checkeo que de lo mismo
user_id= "76561198075035785"
userdata(user_id)

{'Cantidad de dinero gastado': 39.98,
 'Porcentaje de recomendación usuario': 0.0,
 'Cantidad de items': 3}

FUNCION #2:  UserForGenre

In [24]:
user_items.head()

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0
3,76561197970982479,277,40,Deathmatch Classic,0,0
4,76561197970982479,277,50,Half-Life: Opposing Force,0,0


In [25]:
#Elimino los registros donde playtime_forever es igual a 0

user_items_limpio = user_items[user_items['playtime_forever'] != 0]


In [26]:
#Creo DataFrame de Generos a partir del DataFrame steam_games
genre= steam_games[["genres","item_id","Año_estreno"]]
genre.head()

,genres,item_id,Año_estreno
88310,Action,761140,2018
88310,Casual,761140,2018
88310,Indie,761140,2018
88310,Simulation,761140,2018
88310,Strategy,761140,2018


In [27]:
# Combino los DataFrames para obtener información relevante
df_funcion2 = user_items_limpio.merge(genre, on='item_id')
df_funcion2.head()

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks,genres,Año_estreno
0,76561197970982479,277,10,Counter-Strike,6,0,Action,2000
1,doctr,541,10,Counter-Strike,93,0,Action,2000
2,corrupted_soul,115,10,Counter-Strike,108,0,Action,2000
3,WeiEDKrSat,28,10,Counter-Strike,328,0,Action,2000
4,death-hunter,121,10,Counter-Strike,6275,0,Action,2000


In [28]:
#Elimino columnas innecesarias
df_funcion2.drop(["items_count","playtime_2weeks", "item_name","item_id"], axis=1, inplace=True)

In [29]:
# Defino DF para la función agrupando por género y usuario
usuario_mas_horas = df_funcion2.groupby(['genres','user_id'])['playtime_forever'].sum().reset_index()
usuario_mas_horas.head()

,genres,user_id,playtime_forever
0,Action,--000--,139469
1,Action,--ace--,69325
2,Action,--ionex--,38315
3,Action,-2SV-vuLB-Kg,42500
4,Action,-404PageNotFound-,117423


In [30]:
# Defino DF para la función agrupando por género y usuario Año estreno
tiempo_por_anio = df_funcion2.groupby(['Año_estreno','user_id','genres'])['playtime_forever'].sum().reset_index()
tiempo_por_anio.head()

,Año_estreno,user_id,genres,playtime_forever
0,1983,2Ta4,Action,18
1,1983,2Ta4,Adventure,18
2,1983,2Ta4,Casual,18
3,1983,76561197966936422,Action,331
4,1983,76561197966936422,Adventure,331


In [31]:
#Defino la funcion #2 ####
"""
Parametro: 
        genero: (str) : Género a consultar.
Retorna:
        usuario que acumula más horas jugadas para el género dado 
        lista de la acumulación de horas jugadas por año de lanzamiento.
"""
def UserForGenre(genero):
    
        df_genero_deseado = usuario_mas_horas[usuario_mas_horas["genres"] == genero]
    
        if df_genero_deseado.empty:
                return {"Mensaje": f"No hay datos para el género '{genero}'"}


    # Encuentra el usuario que más jugó ese género.
        usuario_mas_jugo = df_genero_deseado.groupby('user_id')["playtime_forever"].sum().idxmax()
    
        horas_jugadas_por_anio = tiempo_por_anio[(tiempo_por_anio["genres"] == genero) & 
                            (tiempo_por_anio["user_id"] == usuario_mas_jugo)].drop(["user_id", "genres"], axis=1)


        retorno = {"Usuario con más horas jugadas para " + genero: usuario_mas_jugo , 
                "Horas jugadas": [{"Año": anio, "Horas": horas} for anio, horas in 
                horas_jugadas_por_anio.reset_index(drop=True).values]}

        return retorno

In [32]:
#Pruebo la función:
UserForGenre("Indie")

{'Usuario con más horas jugadas para Indie': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': '2001', 'Horas': 11},
  {'Año': '2003', 'Horas': 1863},
  {'Año': '2006', 'Horas': 1673},
  {'Año': '2007', 'Horas': 1070},
  {'Año': '2008', 'Horas': 1366},
  {'Año': '2009', 'Horas': 28993},
  {'Año': '2010', 'Horas': 21487},
  {'Año': '2011', 'Horas': 100155},
  {'Año': '2012', 'Horas': 148459},
  {'Año': '2013', 'Horas': 169349},
  {'Año': '2014', 'Horas': 326927},
  {'Año': '2015', 'Horas': 751765},
  {'Año': '2016', 'Horas': 815989},
  {'Año': '2017', 'Horas': 33887}]}

In [33]:
#CHECKEO 
prueba = df_funcion2[(df_funcion2["Año_estreno"] == "2001") & (df_funcion2["user_id"] == "REBAS_AS_F-T")]
prueba.sort_values(by="Año_estreno", ascending=True)



,user_id,playtime_forever,genres,Año_estreno
5226476,REBAS_AS_F-T,11,Action,2001
5226477,REBAS_AS_F-T,11,Free to Play,2001
5226478,REBAS_AS_F-T,11,Indie,2001


FUNCION #3: DEVELOPER   

In [36]:
#Creo DataFrame de games sin genero
games_sin_genre = steam_games.drop("genres", axis=1)
games_sin_genre.head()

,app_name,price,item_id,developer,Año_estreno
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88310,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018


In [37]:
#Ahora van a quedar muchos duplicados porque había registros casi repetidos por el desanidado de la columna genero. Los elimino
df_funcion3 = games_sin_genre.drop_duplicates(keep= "first")

In [38]:
#Defino la funcion #3
"""
Parametros:
        -desarrollador (str): El desarrollador del juego Steam que se ingresa 
Retorna: dataframe:
        -Año
        -Cantidad de items por año: cantidad de juegos publicados por el desarrollador en el año 
        -Porcentaje de juegos free: porcentaje de juegos gratis con respecto a los publicados en ese año
"""
def developer(desarrollador):
    
    #Se filtra el dataframe df_funcion3 para igualarlo al dato que se ingresa
    data= df_funcion3[df_funcion3["developer"]== desarrollador]
    if data.empty:
        return {"message": "Desarrollador no encontrado"}
    #Se agrupa por año para contar los items por año
    cantidad = data.groupby("Año_estreno")["item_id"].count()
    #Se agrupa por price para encontrar la cantidad free
    free_anio= data[data["price"]== 0.0].groupby("Año_estreno")["item_id"].count()
    porcentaje_gratis= (free_anio/cantidad*100).fillna(0).astype(int)
    
    #se crea una salida como dataframe
    tabla= pd.DataFrame({
        "Año": cantidad.index, #indice
        "Cantidad de items por año" : cantidad.values, #valor
        "Porcentaje de juegos free" : porcentaje_gratis.values #valor
    })
    #tabla= tabla.to_dict(orient="records")
    return tabla

In [41]:
#Pruebo la Función:
desarrollador = "Ubisoft - San Francisco"
developer(desarrollador)

,Año,Cantidad de items por año,Porcentaje de juegos free
0,2012,114,0
1,2013,116,0
2,2014,265,0
3,2015,248,0
4,2016,247,0
5,2017,269,0


FUNCION #4: best_developer_year

In [34]:
#Reviso types
#user_items.dtypes
#steam_games.dtypes
#user_reviews.dtypes

In [42]:
# Convertir la columna 'item_id' del DataFrame games a tipo numérico (si está en formato cadena)
user_reviews['item_id'] = user_reviews['item_id'].astype('Int64')

# Convertir la columna 'item_id' del DataFrame user_items a tipo numérico (si está en formato cadena)
user_reviews['item_id'] = user_reviews['item_id'].astype('Int64')



In [43]:
# Realizo el merge de los DataFrames antes de definir la función
df_combinado2 = user_items.merge(steam_games, on='item_id')
df_combinado2 = df_combinado2.merge(user_reviews, on=['user_id', 'item_id'])
df_combinado2.head()

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks,genres,app_name,price,developer,Año_estreno,recommend,sentiment_analisis,posted_year
0,Bennysaputra,87,10,Counter-Strike,24336,6,Action,Counter-Strike,9.99,Valve,2000,1,1,2015
1,76561198040188061,174,10,Counter-Strike,3895,0,Action,Counter-Strike,9.99,Valve,2000,1,2,2011
2,mayshowganmore,5027,10,Counter-Strike,10113,0,Action,Counter-Strike,9.99,Valve,2000,1,1,2014
3,BestinTheWorldThund3r,112,10,Counter-Strike,264,0,Action,Counter-Strike,9.99,Valve,2000,1,2,2014
4,apex124,38,10,Counter-Strike,1558,1,Action,Counter-Strike,9.99,Valve,2000,1,1,None


In [44]:
#Elimino columnas innecesarias
df_funcion4 = df_combinado2.drop(['user_id', 'items_count', 'item_id', 'item_name',
  'playtime_forever','playtime_2weeks', 'genres', 
    'price', 'posted_year'], axis= 1)

In [45]:
#Defino la funcion #4
"""
Parametros:
        -Anio (int): Se ingresa un año
Retorna: 
        Devuelve el top 3 de desarrolladores 
        con juegos MÁS recomendados por usuarios para el año dado.
        (reviews.recommend = True y comentarios positivos)
"""
def best_developer_year(anio):
    # Filtrar el DataFrame combinado para obtener los juegos del año especificado
    juegos_del_anio = df_funcion4[df_funcion4['Año_estreno'] == str(anio)]

    # Filtrar los juegos recomendados (recommend = True y comentarios positivos)
    juegos_recomendados = juegos_del_anio[(juegos_del_anio['recommend'] == True) & (juegos_del_anio['sentiment_analisis'] == 2)]

    if juegos_recomendados.empty:
        return {"Mensaje": f"No hay juegos recomendados para el año {anio}"}

    # Calcular el número de juegos recomendados por desarrollador
    juegos_por_desarrollador = juegos_recomendados.groupby('developer')['app_name'].count().reset_index()

    # Ordenar por número de juegos recomendados en orden descendente
    juegos_por_desarrollador = juegos_por_desarrollador.sort_values(by='app_name', ascending=False).reset_index()

    # Tomar los 3 primeros desarrolladores
    top_desarrolladores = juegos_por_desarrollador.head(3)

    # Construir el resultado en el formato deseado
    resultado = [{"Puesto " + str(i + 1): row['developer']} for i, row in top_desarrolladores.iterrows()]

    return resultado


In [46]:
# Ejemplo de uso:
anio = 2015  # Reemplaza con el año que desees
resultado = best_developer_year(anio)
print(resultado)

[{'Puesto 1': 'Psyonix, Inc.'}, {'Puesto 2': 'Trion Worlds'}, {'Puesto 3': 'Kyle Seeley'}]


FUNCION #5 : developer_reviews_analysis

In [47]:
#Defino el DataFrame que voy a usar conservando las columnas que necesito
df_funcion5 = df_combinado2[["developer", "sentiment_analisis"]]
df_funcion5.head()

,developer,sentiment_analisis
0,Valve,1
1,Valve,2
2,Valve,1
3,Valve,2
4,Valve,1


In [48]:
#Defino la funcion #5
"""
Parametros:
        -desarrolladora (str): Se ingresa una desarrolladora 
Retorna: Diccionario
        Devuelve un diccionario con el nombre del desarrollador como llave 
        y una lista con la cantidad total de registros de reseñas de usuarios que se 
        encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
"""
def developer_reviews_analysis(desarrolladora):
    # Filtrar el DataFrame df_funcion5 por el desarrollador y por análisis de sentimiento positivo o negativo
    desarrolladora_reviews = df_funcion5[(df_funcion5['developer'] == desarrolladora) & ((df_funcion5['sentiment_analisis'] == 0) | (df_funcion5['sentiment_analisis'] == 2))]

    if desarrolladora_reviews.empty:
        return {desarrolladora: {'Positive': 0, 'Negative': 0}}

    # Contar la cantidad de reseñas positivas y negativas
    count_positive = (desarrolladora_reviews['sentiment_analisis'] == 2).sum()
    count_negative = (desarrolladora_reviews['sentiment_analisis'] == 0).sum()

    # Crear el diccionario de retorno
    resultado = {desarrolladora: ["Negative = " + str(count_negative), "Positive = " + str(count_positive)]}

    return resultado


In [49]:
# Ejemplo de uso:
desarrolladora = 'Valve'  # Reemplaza con el nombre del desarrollador que desees
resultado = developer_reviews_analysis(desarrolladora)
print(resultado)

{'Valve': ['Negative = 675', 'Positive = 2442']}


Creación de archivos
Se transforman los DataFrames de las funciones a formato parquet

In [50]:
df_funcion1.to_parquet("Data/df_funcion1.parquet")
#df_funcion2.to_parquet("Data/df_funcion2.parquet") #No lo uso porque es muy pesado para subir a GH
usuario_mas_horas.to_parquet("Data/df_funcion2_A.parquet")
tiempo_por_anio.to_parquet("Data/df_funcion2_B.parquet")

df_funcion3.to_parquet("Data/df_funcion3.parquet")
df_funcion4.to_parquet("Data/df_funcion4.parquet")
df_funcion5.to_parquet("Data/df_funcion5.parquet")

recommend.to_parquet("Data/recommend.parquet")